In [1]:
from datetime import datetime

import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [2]:
! ls data/

T1.csv


In [3]:
sc = SparkSession.builder.appName('Cheatsheet').getOrCreate()

21/07/29 22:57:27 WARN Utils: Your hostname, sid-HP-240-G3-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.43.10 instead (on interface wlp9s0f0)
21/07/29 22:57:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/29 22:57:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Reading CSV

In [4]:
# Reading CSV in pandas
df = pd.read_csv('data/T1.csv')

# Reading CSV in PySpark
spark_df = sc.read.options(header=True) \
                  .csv("data/T1.csv")

In [5]:
df.head()

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


In [6]:
spark_df.show(5)

+----------------+-------------------+----------------+-----------------------------+------------------+
|       Date/Time|LV ActivePower (kW)|Wind Speed (m/s)|Theoretical_Power_Curve (KWh)|Wind Direction (°)|
+----------------+-------------------+----------------+-----------------------------+------------------+
|01 01 2018 00:00|   380.047790527343|5.31133604049682|             416.328907824861|  259.994903564453|
|01 01 2018 00:10|    453.76919555664|5.67216682434082|             519.917511061494|   268.64111328125|
|01 01 2018 00:20|   306.376586914062|5.21603679656982|             390.900015810951|  272.564788818359|
|01 01 2018 00:30|   419.645904541015|5.65967416763305|             516.127568975674|  271.258087158203|
|01 01 2018 00:40|   380.650695800781|5.57794094085693|             491.702971953588|  265.674285888671|
+----------------+-------------------+----------------+-----------------------------+------------------+
only showing top 5 rows



In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50530 entries, 0 to 50529
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Date/Time                      50530 non-null  object 
 1   LV ActivePower (kW)            50530 non-null  float64
 2   Wind Speed (m/s)               50530 non-null  float64
 3   Theoretical_Power_Curve (KWh)  50530 non-null  float64
 4   Wind Direction (°)             50530 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.9+ MB


In [8]:
spark_df.printSchema()

root
 |-- Date/Time: string (nullable = true)
 |-- LV ActivePower (kW): string (nullable = true)
 |-- Wind Speed (m/s): string (nullable = true)
 |-- Theoretical_Power_Curve (KWh): string (nullable = true)
 |-- Wind Direction (°): string (nullable = true)



### Nerdy Tip

* **pandas** uses auto infer mechanism by default while in **PySpark** the `inferSchema` is set to `False` by default.


* While reading the CSV in pandas use `parse_dates` parameter for converting the column type to `datetime`.

## Infer dtypes while reading CSV

### pandas

In [9]:
custom_date_parser = lambda x: datetime.strptime(x, "%d %m %Y %H:%M")

df = pd.read_csv('data/T1.csv',
                 parse_dates=['Date/Time'],
                 date_parser=custom_date_parser
                )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50530 entries, 0 to 50529
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Date/Time                      50530 non-null  datetime64[ns]
 1   LV ActivePower (kW)            50530 non-null  float64       
 2   Wind Speed (m/s)               50530 non-null  float64       
 3   Theoretical_Power_Curve (KWh)  50530 non-null  float64       
 4   Wind Direction (°)             50530 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 1.9 MB


### PySpark

In [10]:
spark_df = sc.read.options(
    header=True,
    inferSchema=True,
    timestampFormat="dd MM y HH:mm"
).csv("data/T1.csv")

spark_df.printSchema()

root
 |-- Date/Time: timestamp (nullable = true)
 |-- LV ActivePower (kW): double (nullable = true)
 |-- Wind Speed (m/s): double (nullable = true)
 |-- Theoretical_Power_Curve (KWh): double (nullable = true)
 |-- Wind Direction (°): double (nullable = true)



In [11]:
df.head()

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,2018-01-01 00:00:00,380.047791,5.311336,416.328908,259.994904
1,2018-01-01 00:10:00,453.769196,5.672167,519.917511,268.641113
2,2018-01-01 00:20:00,306.376587,5.216037,390.900016,272.564789
3,2018-01-01 00:30:00,419.645905,5.659674,516.127569,271.258087
4,2018-01-01 00:40:00,380.650696,5.577941,491.702972,265.674286


In [12]:
spark_df.show(5)

+-------------------+-------------------+----------------+-----------------------------+------------------+
|          Date/Time|LV ActivePower (kW)|Wind Speed (m/s)|Theoretical_Power_Curve (KWh)|Wind Direction (°)|
+-------------------+-------------------+----------------+-----------------------------+------------------+
|2018-01-01 00:00:00|   380.047790527343|5.31133604049682|             416.328907824861|  259.994903564453|
|2018-01-01 00:10:00|    453.76919555664|5.67216682434082|             519.917511061494|   268.64111328125|
|2018-01-01 00:20:00|   306.376586914062|5.21603679656982|             390.900015810951|  272.564788818359|
|2018-01-01 00:30:00|   419.645904541015|5.65967416763305|             516.127568975674|  271.258087158203|
|2018-01-01 00:40:00|   380.650695800781|5.57794094085693|             491.702971953588|  265.674285888671|
+-------------------+-------------------+----------------+-----------------------------+------------------+
only showing top 5 rows



## Renaming columns

In [13]:
custom_date_parser = lambda x: datetime.strptime(x, "%d %m %Y %H:%M")

df = pd.read_csv('data/T1.csv',
                 parse_dates=['Date/Time'],
                 date_parser=custom_date_parser
                )

spark_df = sc.read.options(
    header=True, 
    inferSchema=True,
    timestampFormat="dd MM y HH:mm"
).csv("data/T1.csv")

### pandas

In [14]:
df = df.rename(columns={
    "Date/Time": "timestamp",
    "LV ActivePower (kW)": "active_pwr",
    "Wind Speed (m/s)": "wind_speed",
    "Theoretical_Power_Curve (KWh)": "theoretical_pwr",
    "Wind Direction (°)": "wind_dir"
})

df.head()

,timestamp,active_pwr,wind_speed,theoretical_pwr,wind_dir
0,2018-01-01 00:00:00,380.047791,5.311336,416.328908,259.994904
1,2018-01-01 00:10:00,453.769196,5.672167,519.917511,268.641113
2,2018-01-01 00:20:00,306.376587,5.216037,390.900016,272.564789
3,2018-01-01 00:30:00,419.645905,5.659674,516.127569,271.258087
4,2018-01-01 00:40:00,380.650696,5.577941,491.702972,265.674286


### PySpark

In [15]:
spark_df = spark_df.withColumnRenamed("Date/Time", "timestamp") \
.withColumnRenamed("LV ActivePower (kW)", "active_pwr") \
.withColumnRenamed("Wind Speed (m/s)", "wind_speed") \
.withColumnRenamed("Theoretical_Power_Curve (KWh)", "theoretical_pwr") \
.withColumnRenamed("Wind Direction (°)", "wind_dir")

spark_df.show(5)

+-------------------+----------------+----------------+----------------+----------------+
|          timestamp|      active_pwr|      wind_speed| theoretical_pwr|        wind_dir|
+-------------------+----------------+----------------+----------------+----------------+
|2018-01-01 00:00:00|380.047790527343|5.31133604049682|416.328907824861|259.994903564453|
|2018-01-01 00:10:00| 453.76919555664|5.67216682434082|519.917511061494| 268.64111328125|
|2018-01-01 00:20:00|306.376586914062|5.21603679656982|390.900015810951|272.564788818359|
|2018-01-01 00:30:00|419.645904541015|5.65967416763305|516.127568975674|271.258087158203|
|2018-01-01 00:40:00|380.650695800781|5.57794094085693|491.702971953588|265.674285888671|
+-------------------+----------------+----------------+----------------+----------------+
only showing top 5 rows



## Changing data type of columns

In [16]:
custom_date_parser = lambda x: datetime.strptime(x, "%d %m %Y %H:%M")

df = pd.read_csv('data/T1.csv',
                 parse_dates=['Date/Time'],
                 date_parser=custom_date_parser
                )

spark_df = sc.read.options(
    header=True,
    inferSchema=True,
    timestampFormat="dd MM y HH:mm"
).csv("data/T1.csv")

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50530 entries, 0 to 50529
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Date/Time                      50530 non-null  datetime64[ns]
 1   LV ActivePower (kW)            50530 non-null  float64       
 2   Wind Speed (m/s)               50530 non-null  float64       
 3   Theoretical_Power_Curve (KWh)  50530 non-null  float64       
 4   Wind Direction (°)             50530 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 1.9 MB


In [18]:
spark_df.printSchema()

root
 |-- Date/Time: timestamp (nullable = true)
 |-- LV ActivePower (kW): double (nullable = true)
 |-- Wind Speed (m/s): double (nullable = true)
 |-- Theoretical_Power_Curve (KWh): double (nullable = true)
 |-- Wind Direction (°): double (nullable = true)



In [19]:
df['LV ActivePower (kW)'] = df['LV ActivePower (kW)'].astype(int)

spark_df = spark_df.withColumn("LV ActivePower (kW)", F.col("LV ActivePower (kW)").cast(IntegerType()))

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50530 entries, 0 to 50529
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Date/Time                      50530 non-null  datetime64[ns]
 1   LV ActivePower (kW)            50530 non-null  int64         
 2   Wind Speed (m/s)               50530 non-null  float64       
 3   Theoretical_Power_Curve (KWh)  50530 non-null  float64       
 4   Wind Direction (°)             50530 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 1.9 MB


In [21]:
spark_df.printSchema()

root
 |-- Date/Time: timestamp (nullable = true)
 |-- LV ActivePower (kW): integer (nullable = true)
 |-- Wind Speed (m/s): double (nullable = true)
 |-- Theoretical_Power_Curve (KWh): double (nullable = true)
 |-- Wind Direction (°): double (nullable = true)



In [22]:
df.head()

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,2018-01-01 00:00:00,380,5.311336,416.328908,259.994904
1,2018-01-01 00:10:00,453,5.672167,519.917511,268.641113
2,2018-01-01 00:20:00,306,5.216037,390.900016,272.564789
3,2018-01-01 00:30:00,419,5.659674,516.127569,271.258087
4,2018-01-01 00:40:00,380,5.577941,491.702972,265.674286


In [23]:
spark_df.show(5)

+-------------------+-------------------+----------------+-----------------------------+------------------+
|          Date/Time|LV ActivePower (kW)|Wind Speed (m/s)|Theoretical_Power_Curve (KWh)|Wind Direction (°)|
+-------------------+-------------------+----------------+-----------------------------+------------------+
|2018-01-01 00:00:00|                380|5.31133604049682|             416.328907824861|  259.994903564453|
|2018-01-01 00:10:00|                453|5.67216682434082|             519.917511061494|   268.64111328125|
|2018-01-01 00:20:00|                306|5.21603679656982|             390.900015810951|  272.564788818359|
|2018-01-01 00:30:00|                419|5.65967416763305|             516.127568975674|  271.258087158203|
|2018-01-01 00:40:00|                380|5.57794094085693|             491.702971953588|  265.674285888671|
+-------------------+-------------------+----------------+-----------------------------+------------------+
only showing top 5 rows

